## Utilizando Web-Scraping para fazer pesquisa de preço

#### Passo 1: instalar bibliotecas

In [1]:
# Remova o comentário(#) da linha abaixo caso esteja executando este script pela primeira vez
#!pip install requests
#!pip install beautifulsoup4

#### Passo 2: Importar bibliotecas

In [51]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### Passo 3: Formatar link de busca

In [52]:
# Copie o modelo do link de busca
url_de_busca = 'https://lista.mercadolivre.com.br/'

# Informe a palvra chave para busca
palavra_chave = 'smart tv 70'

# Formata a palavra chave conforme modelo do site de busca
palavra_chave = palavra_chave.replace(' ','-')

# Formata link
link = url_de_busca+palavra_chave
print(link)

https://lista.mercadolivre.com.br/smart-tv-70


#### Passo 4: Obter informações do link

In [53]:
# Requisita informações do site
response = requests.get(link)

# Verifica status de acesso
print('Código de status:', response.status_code) 

Código de status: 403


Esse **403 Forbidden** acontece porque o Mercado Livre (assim como outros grandes e-commerces) possui mecanismos de proteção contra bots e scrapers simples, e o requests.get() sem nenhum cabeçalho já denuncia que não é um navegador “de verdade”.

#### 4.1 Usar um User-Agent válido

Alguns sites **bloqueiam requisições que não parecem vir de navegadores**. Então você pode **enviar cabeçalhos simulando um browser**.

##### Para criar um User-Agent (real)
- Identifique qual é a versão do seu navegador acessando, por exemplo, a URL chrome://settings/help
- Crie um cabeçalho com a versão do seu navegador.

In [54]:
# Cabeçalho com User-Agent
headers = {'User-Agent': 'Chrome/138.0.7204.184'}
#headers = {'User-Agent': '0'}

# Requisita informações do site
response = requests.get(link, headers=headers)

# Verifica status de acesso
print("Código de status:", response.status_code)

Código de status: 200


In [55]:
# Cria um objeto BeautifulSoup com as informações html
soup = BeautifulSoup(response.content, 'html.parser')

#### Passo 5: Encontrar elementos

In [56]:
# Encontra as informações contidas na tag de resultado de busca
resultados = soup.find_all(class_ = "ui-search-layout__item")
# Conta quantos resultados apareceram na primeira página
len(resultados)

0

#### Passo 6: Visualizar links dos produtos

In [57]:
# Criar uma lista vazia para guardar dados
lista_de_links = []

# Criar laço de repetição para filtrar todos os dados
for i in resultados:
    lista_de_links.append(i.find('a')['href'])

# Visualizar links
for j in lista_de_links:
    print(j)

#### Passo 7: Extrarir informações de cada produto listado

In [58]:
# Requisita informações do link do produto e cria objeto BeautifulSoup
produto = BeautifulSoup(requests.get(lista_de_links[0],headers=headers).content, 'html.parser')

IndexError: list index out of range

In [10]:
# Seleciona o título
produto.find(class_ = 'ui-pdp-title').text

'Smart Tv 58 Philco Led 4k Google Tv Hdr10 P58kga'

In [11]:
# Seleciona o preço
produto.find(class_ = 'andes-money-amount__fraction').text

'3.284'

In [12]:
# Seleciona a quantidade de vendas
produto.find(class_ = 'ui-pdp-subtitle').text

'Novo  |  +5mil vendidos'

In [13]:
# Seleciona a descrição
produto.find(class_ = 'ui-pdp-description__content').text

'A Smart TV Philco 58” 4K UHD P58KGA com Google TV foi desenvolvida para quem busca aliar alta qualidade de imagem, conectividade inteligente e excelente custo-benefício. Com tela de 58 polegadas, resolução Ultra HD e suporte a HDR10, ela proporciona uma experiência visual rica em detalhes, cores vibrantes e contraste superior — ideal para filmes, séries, esportes e games. Seu design moderno com bordas finas confere sofisticação ao ambiente, seja na sala de estar, quarto ou até em espaços comerciais.Equipada com o sistema operacional Google TV, a P58KGA oferece uma navegação fluida, intuitiva e altamente personalizada. O acesso direto a aplicativos como Netflix, YouTube, Prime Video e Globoplay, além de milhares de outros via Google Play, transforma a televisão em um verdadeiro hub de entretenimento. Com controle remoto por voz, comandos integrados ao Google Assistente e Chromecast embutido, o usuário tem liberdade total para interagir com a TV sem tocar em nenhum botão.Outro destaque 

#### Criar tabela para armazenar dados

In [14]:
# Criar o nome de cada coluna
colunas = ['Título','Preço','Quantidade de Vendas','Descrição do produto']

# Criar a tabela com as colunas informadas
tabela = pd.DataFrame(columns=colunas)

# Visualizar tabela
tabela

,Título,Preço,Quantidade de Vendas,Descrição do produto


In [15]:
# Criar laço de repetição para armazenar as informações de todos os produtos da lista de links

id = 0 # Contador
for j in lista_de_links:
    try:
        # Requisita informações do link do produto e cria objeto BeautifulSoup
        produto = BeautifulSoup(requests.get(j, headers=headers).content, 'html.parser')
    
        # Seleciona o título
        titulo = produto.find(class_ = 'ui-pdp-title').text
        # Seleciona o preço
        preco = produto.find(class_ = 'andes-money-amount__fraction').text
        # Seleciona a quantidade de vendas
        vendidos = produto.find(class_ = 'ui-pdp-subtitle').text
        # Seleciona a descrição
        descricao = produto.find(class_ = 'ui-pdp-description__content').text

        # Insere uma nova linha na tabela com as informações do produto
        tabela.loc[len(tabela)] = [titulo,preco,vendidos,descricao]

    except:
        print(f'Houve um problema no link de índice: {id}')
    
    # Incrementa uma posição no contador
    id = id+1

Houve um problema no link de índice: 11
Houve um problema no link de índice: 12


In [16]:
# Visualizar tabela preenchida
tabela

,Título,Preço,Quantidade de Vendas,Descrição do produto
0,Smart Tv 58 Philco Led 4k Google Tv Hdr10 P58kga,3.284,Novo | +5mil vendidos,A Smart TV Philco 58” 4K UHD P58KGA com Google...
1,Smart TV 50” Philco PTV50VA4REGB Roku TV LED D...,2.299,Novo | +10mil vendidos,A Smart TV Philco PTV50VA4REGB é a escolha per...
2,Smart Tv Philips 70 4k Google Tv Comando De Vo...,4.999,Novo | +100 vendidos,Smart TV Philips Série 7019 - Google TV: A pl...
3,Smart TV 70'' Crystal UHD 4K 70DU8000 2024 Sam...,5.799,Novo | +1000 vendidos,"FINA, ELEGANTE E INTELIGENTE: NUNCA FOI POSSÍV..."
4,Smart Tv 70'' 70pug7408/78 70 4k Google Tv Phi...,4.799,Novo | +1000 vendidos,IMAGENS VIBRANTES SEMPRE!A Smart TV Philips 70...
5,Smart Tv 70'' 4k Uhd 70Ur8750 Thinq Ai LG Bilvot,5.499,Novo | +500 vendidos,Smart TV 70'' 4K UHD UR8750 ThinQ AI LG Bilvot...
6,Smart Tv Aoc Roku Led 50'' 4k 50u6125/78g Hdmi...,2.489,Novo | 4 vendidos,"Somos a Mega Byte, loja física e online que at..."
7,Smart Tv Philips 50 4k 50pug7019 Google Tv Com...,2.749,Novo | +10mil vendidos,Smart TV Philips Série 7019 - Google TV: A pl...
8,Smart Tv Multilaser Tl052m 32 Led Roku Tv,1.230,Novo,"- PRODUTO COM ESTOQUE NO BRASIL, ENVIO IMEDIAT..."
9,Samsung Smart Tv 70 Crystal Uhd 4k U8500f 2025...,5.276,Novo | +5 vendidos,"Samsung Smart TV 70"" Crystal UHD 4K U8500F 202..."


#### Passo 8: Salvar tabela em uma planilha de Excel

In [17]:
# Salvar DataFrame
tabela.to_excel('Lista de produtos.xlsx')

### Exercício 1: Encontrar uma solução para listar todos os resultados quando houver 2 ou mais páginas de resultado

In [18]:
# Copie o modelo do link de busca
url_de_busca = 'https://lista.mercadolivre.com.br/'

# Informe a palvra chave para busca
palavra_chave = 'celular iphone'

# Formata a palavra chave conforme modelo do site de busca
palavra_chave = palavra_chave.replace(' ','-')

# Formata link
link = url_de_busca+palavra_chave
print(link)

https://lista.mercadolivre.com.br/celular-iphone


In [20]:
# Cabeçalho com User-Agent
headers = {'User-Agent': 'Chrome/138.0.7204.184'}
#headers = {'User-Agent': '0'}

# Requisita informações do site
response = requests.get(link, headers=headers)

# Verifica status de acesso
print("Código de status:", response.status_code)

Código de status: 200


In [21]:
# Cria um objeto BeautifulSoup com as informações html
soup = BeautifulSoup(response.content, 'html.parser')

In [27]:
# Encontra as informações contidas na tag de resultado de busca
resultados = soup.find_all(class_ = "ui-search-layout__item")
# Conta quantos resultados apareceram na primeira página
len(resultados)

52

In [37]:
# Criar uma lista vazia para guardar dados
lista_de_links = []

# Criar laço de repetição para filtrar todos os dados
for i in resultados:
    lista_de_links.append(i.find('a')['href'])

# Visualizar links
for j in lista_de_links:
    print(j)

https://click1.mercadolivre.com.br/mclics/clicks/external/MLB/count?a=Lc9%2BFfjt9ZMonzTi05QmelBY1zwgZd9byN5gwWymakffgGkOh1IBGR73Ljwx7osV2TjeKjA5J%2BE5v%2BdH3nC0546iP8uANtlpLc1QRK3G7WS0fD3OVRMErjkOpLzK5p4lWWEINRUE%2FNiXPwjqp5OD4Eihr5Cjv8cmDOcJE2vptUQZU9JqEuG1uobgJg9wgeHJ%2BwblVB2ed7cUhVfCN%2FXhkCabse6n%2FpbcjDUcMQqvF2pr6n16uuNdYXl8%2Fgu%2BMFJlYShgX%2B2YcBHSI9AsjsM1DVECl7W7G0nI5uY1WB1H0RLyECCQeFQKmNHmXBIG8aVjtSAnTxoZeaH%2BtD6cuTGayXSRvv0CEjuM8LTrhjXItu35RhmPoWE5cJ1KS9ehhVJ6we%2BLtZpgrBVllWSS233AgfzvqnfdicnHXv69q6220Xyom8BrJ1TZbj15k7x%2F5SuZ0sd4Hk9cyBmSDzz1Fa93s8U8EPjj1%2FMWnOMRT4irAjKhzRoSRLBGXcQAabLJHd1knkVkwfTqn0oqyYruizb60Ss%2B8hU1xHgP57VgvgqSto0O6%2BPzchHhuEJVdrew5Y20UxpgV%2Fun9m%2B528M%2FSxvHIGywwPVpbo3z5%2FxaFYotG5jUKa567peHIqvUOomXTTYwLk4I8P5DQQ%2F0DlUXGnBqeER18I77u86DWzbyA46WKtNQXLJ8O7NpVDs8NAvz3fBWjz%2FvLEeY0EBvULIF09vt4woL52F13mHrW66qx0mrYlnqalgsbxfTICmj6XMgjBKR8w4TaAbMgU1iI6UzdEIuLBqcL8KcI5cavfeQKQmIxLPG6XSqn9fEfxckS%2FEurhPLo61SkHM2kSTxGECpODCpJ6o2X%2FTFOdtQIHPbST8%3D&searchV

In [38]:
# Criar o nome de cada coluna
colunas = ['Título','Preço','Quantidade de Vendas','Descrição do produto']

# Criar a tabela com as colunas informadas
tabela = pd.DataFrame(columns=colunas)

# Visualizar tabela
tabela

,Título,Preço,Quantidade de Vendas,Descrição do produto


In [42]:
# Criar laço de repetição para armazenar as informações de todos os produtos da lista de links

id = 0 # Contador
for j in lista_de_links:
    try:
        # Requisita informações do link do produto e cria objeto BeautifulSoup
        produto = BeautifulSoup(requests.get(j, headers=headers).content, 'html.parser')

        # Seleciona o título
        titulo = produto.find(class_ = 'ui-pdp-title').text      
        # Seleciona o preço
        preco = produto.find(class_ = 'andes-money-amount__fraction').text
        # Seleciona a quantidade de vendas
        vendidos = produto.find(class_ = 'ui-pdp-subtitle').text
        # Seleciona a descrição
        descricao = produto.find(class_ = 'ui-pdp-description__content').text

        # Insere uma nova linha na tabela com as informações do produto
        tabela.loc[len(tabela)] = [titulo,preco,vendidos,descricao]

    except Exception as e:
        print(f'Houve um problema no link de índice: {id}. Erro: {e}')
    
    # Incrementa uma posição no contador
    id = id+1

Não foi possível obter o título do link 0
Houve um problema no link de índice: 0. Erro: 'NoneType' object has no attribute 'text'
Não foi possível obter o título do link 1
Houve um problema no link de índice: 1. Erro: 'NoneType' object has no attribute 'text'
Não foi possível obter o título do link 2
Houve um problema no link de índice: 2. Erro: 'NoneType' object has no attribute 'text'
Não foi possível obter o título do link 3
Houve um problema no link de índice: 3. Erro: 'NoneType' object has no attribute 'text'
Não foi possível obter o título do link 4
Houve um problema no link de índice: 4. Erro: 'NoneType' object has no attribute 'text'
Não foi possível obter o título do link 5
Houve um problema no link de índice: 5. Erro: 'NoneType' object has no attribute 'text'
Não foi possível obter o título do link 6
Houve um problema no link de índice: 6. Erro: 'NoneType' object has no attribute 'text'
Não foi possível obter o título do link 7
Houve um problema no link de índice: 7. Erro: 'N

In [34]:
# Visualizar tabela preenchida
tabela

,Título,Preço,Quantidade de Vendas,Descrição do produto
0,Título não encontrado,Preço não encontrado,Quantidade de vendas não encontrada,Descrição não encontrada
1,Título não encontrado,Preço não encontrado,Quantidade de vendas não encontrada,Descrição não encontrada
2,Título não encontrado,Preço não encontrado,Quantidade de vendas não encontrada,Descrição não encontrada
3,Título não encontrado,Preço não encontrado,Quantidade de vendas não encontrada,Descrição não encontrada
4,Título não encontrado,Preço não encontrado,Quantidade de vendas não encontrada,Descrição não encontrada
5,Título não encontrado,Preço não encontrado,Quantidade de vendas não encontrada,Descrição não encontrada
6,Título não encontrado,Preço não encontrado,Quantidade de vendas não encontrada,Descrição não encontrada
7,Título não encontrado,Preço não encontrado,Quantidade de vendas não encontrada,Descrição não encontrada
8,Título não encontrado,Preço não encontrado,Quantidade de vendas não encontrada,Descrição não encontrada
9,Título não encontrado,Preço não encontrado,Quantidade de vendas não encontrada,Descrição não encontrada


### Exercício 2 (extra): Formatar os dados da tabela